In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_columns=1000
pd.options.display.max_rows=1000
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from scipy import stats
import itertools

In [2]:
import multiprocessing as mp
import time

In [3]:
def parse_dict_of_dict(_dict, _str = ''):
    ret_dict = {}
    for k, v in _dict.items():
        if isinstance(v, dict):
            ret_dict.update(parse_dict_of_dict(v, _str= '_'.join([_str, k]).strip('_')))
        elif isinstance(v, list):
            for index, itemv in enumerate(v):
                if isinstance(item, dict):
                    ret_dict.update(parse_dict_of_dict(item,  _str= '_'.join([_str, k, str(index)]).strip('_')))
                else:
                    ret_dict['_'.join([_str, k, str(index)]).strip('_')] = item
        else:
            try:
                ret_dict['_'.join([_str, k]).strip('_')] = str(v)
            except Exception as e:
                ret_dict['_'.join([_str, k]).strip('_')] = unicode.encode(v, errors='ignore')
    return ret_dict

In [4]:
def get_mode(x):
    try:
        return x.value_counts().index[0]
    except:
        return np.nan
    
def get_mode_count(x):
    try:
        return x.value_counts().values[0]
    except:
        return np.nan

In [5]:
view_log = pd.read_csv('view_log.csv')
view_log = view_log.drop_duplicates()
view_log['server_time'] = pd.to_datetime(view_log.server_time)
view_log = view_log.merge(pd.read_csv('item_data.csv'), on='item_id', how='left')
view_log.sort_values(by=['user_id', 'server_time'], inplace=True)
view_log.shape
view_log.head()

(3017496, 10)

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
158742,2018-10-18 14:27:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0
159389,2018-10-18 14:43:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0
159502,2018-10-18 14:46:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0
159573,2018-10-18 14:47:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0
328522,2018-10-22 09:48:00,android,295495,0,40228,332.0,7.0,24.0,126.0,8351.0


In [6]:
# view_log.device_type.unique()

In [7]:
train = pd.read_csv('train.csv')
train['impression_time'] = pd.to_datetime(train.impression_time)
train.shape

test = pd.read_csv('test.csv')
test['impression_time'] = pd.to_datetime(test.impression_time)
test.shape

train.head()

(237609, 7)

(90675, 6)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [8]:
# train.drop(columns='impression_id').drop_duplicates().shape
# test.drop(columns='impression_id').drop_duplicates().shape

In [9]:
# view_log.server_time.min(), view_log.server_time.max()
# train.impression_time.min(), train.impression_time.max()
# test.impression_time.min(), test.impression_time.max()

In [10]:
df = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)
df.shape
df.head()

(328284, 7)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0.0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1.0
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0.0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0.0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0.0


In [11]:
df.sort_values(by=['user_id','impression_time'], inplace=True)

In [12]:
grpby = view_log.groupby(['session_id', 'user_id'])
session_vars = grpby['server_time'].agg(start_time='min',  end_time='max', no_of_logs='count').reset_index()
session_vars.sort_values(by = ['user_id', 'start_time'], inplace=True)
session_vars['duration'] = (session_vars.end_time - session_vars.start_time).dt.seconds
session_vars['time_since_last_session'] = ((session_vars.start_time - session_vars.end_time.shift(1)) * (session_vars.user_id == session_vars.user_id.shift(1)).astype(int)).dt.seconds
session_vars['time_for_next_session'] = ((session_vars.start_time.shift(-1) - session_vars.end_time) * (session_vars.user_id == session_vars.user_id.shift(-1)).astype(int)).dt.seconds

In [13]:
view_log = view_log.merge(session_vars)
view_log.shape
view_log.head()

(3017496, 16)

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type,start_time,end_time,no_of_logs,duration,time_since_last_session,time_for_next_session
0,2018-10-18 14:27:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
1,2018-10-18 14:43:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
2,2018-10-18 14:46:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
3,2018-10-18 14:47:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
4,2018-10-22 09:48:00,android,295495,0,40228,332.0,7.0,24.0,126.0,8351.0,2018-10-22 09:48:00,2018-10-22 09:48:00,1,0,68460.0,83940.0


In [14]:
# view_log.duration.mean()

In [15]:
df.shape, view_log.shape

((328284, 7), (3017496, 16))

In [16]:
df_ = df[['user_id', 'impression_id', 'impression_time']].merge(view_log, on='user_id')
df_.shape
df_ = df_[df_.impression_time > df_.server_time]
# df_ = df_[df_.impression_time > df_.end_time]
df_.shape

(14428129, 18)

(12034101, 18)

In [17]:
df_['time_delta_in_days'] = (df_.impression_time - df_.server_time).dt.days

In [18]:
df_.head()

,user_id,impression_id,impression_time,server_time,device_type,session_id,item_id,item_price,category_1,category_2,category_3,product_type,start_time,end_time,no_of_logs,duration,time_since_last_session,time_for_next_session,time_delta_in_days
0,0,5f98c2c31a8006e510448c02ec74d50f,2018-11-26 23:30:00,2018-10-18 14:27:00,android,268465,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0,39
1,0,5f98c2c31a8006e510448c02ec74d50f,2018-11-26 23:30:00,2018-10-18 14:43:00,android,268465,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0,39
2,0,5f98c2c31a8006e510448c02ec74d50f,2018-11-26 23:30:00,2018-10-18 14:46:00,android,268465,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0,39
3,0,5f98c2c31a8006e510448c02ec74d50f,2018-11-26 23:30:00,2018-10-18 14:47:00,android,268465,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0,39
4,0,5f98c2c31a8006e510448c02ec74d50f,2018-11-26 23:30:00,2018-10-22 09:48:00,android,295495,40228,332.0,7.0,24.0,126.0,8351.0,2018-10-22 09:48:00,2018-10-22 09:48:00,1,0,68460.0,83940.0,35


In [19]:
df_.sort_values(by=['impression_id', 'server_time'], inplace=True)

#### Items Aggregate

In [20]:
def get_mode(x):
    try:
        return x.value_counts().index[0]
    except:
        return np.nan
    
df_y = df_.drop_duplicates(subset=['impression_id', 'item_id'], keep='last').groupby('impression_id').agg({
    'item_id': ['count'],
    'item_price':['min', 'max', 'mean','std', ('range', lambda x : x.max() - x.min())],
    'category_1': [('mode', get_mode), 'nunique'],
    'category_2': [('mode', get_mode), 'nunique'],
    'category_3': [('mode', get_mode), 'nunique'],
    'product_type': [('mode', get_mode), 'nunique'],
    
    # mode count can be added
})
df_y.columns = ['_'.join(col).strip('_') for col in df_y.columns.values]
df_y = df_y.reset_index()
df_y.shape
df_y.columns = list(map(lambda x: 'hist_' + x if x != 'impression_id' else x, df_y.columns ))
df_y.head()

(326663, 15)

,impression_id,hist_item_id_count,hist_item_price_min,hist_item_price_max,hist_item_price_mean,hist_item_price_std,hist_item_price_range,hist_category_1_mode,hist_category_1_nunique,hist_category_2_mode,hist_category_2_nunique,hist_category_3_mode,hist_category_3_nunique,hist_product_type_mode,hist_product_type_nunique
0,00003e3b9e5336685200ae85d21b4f5e,4,249.0,2298.0,1190.250000,959.315859,2049.0,13.0,3,18.0,3,117.0,4,6379.0,4
1,000053b1e684c9e7ea73727b2238ce18,3,4416.0,18496.0,9623.333333,7722.538529,14080.0,13.0,3,18.0,3,5.0,3,2088.0,3
2,00005d011db80a956aab176cc94d1d37,26,352.0,58307.0,6973.230769,13328.264827,57955.0,9.0,13,9.0,20,292.0,21,8898.0,26
3,0000b2815cc3c2b56867cbbf4d36efa5,13,723.0,40642.0,11765.000000,11633.737705,39919.0,9.0,7,8.0,11,159.0,12,4153.0,13
4,000109e6bf54b3b20e8547b6026bc355,14,844.0,101235.0,20135.928571,29601.888530,100391.0,12.0,9,57.0,12,223.0,13,9015.0,14


In [21]:
df_y.to_csv('item_agg_vars.csv', index=False)

In [22]:
# df_y = df_[df_.time_delta_in_days >= 7].drop_duplicates(subset=['impression_id', 'item_id'], keep='last').groupby('impression_id').agg({
#     'item_id': ['count'],
#     'item_price':['min', 'max', 'mean','std', ('range', lambda x : x.max() - x.min())],
#     'category_1': [('mode', get_mode), 'nunique'],
#     'category_2': [('mode', get_mode), 'nunique'],
#     'category_3': [('mode', get_mode), 'nunique'],
#     'product_type': [('mode', get_mode), 'nunique'],
# })
# df_y.columns = ['_'.join(col).strip('_') for col in df_y.columns.values]
# df_y = df_y.reset_index()
# df_y.shape
# df_y.columns = list(map(lambda x: 'hist_7d_' + x if x != 'impression_id' else x, df_y.columns ))
# df_y.head()

In [23]:
# df_y.to_csv('item_agg_vars_7d.csv', index=False)

#### Last Item Viewed

In [24]:
df1 = df_.drop_duplicates(subset=['impression_id'], keep='last')
df1 = df1[['impression_id', 'server_time', 'device_type', 'item_id', 'item_price',
     'category_1', 'category_2', 'category_3', 'product_type']]
df1.columns = list(map(lambda x: 'last_' + x if x != 'impression_id' else x, df1.columns ))
df1.shape
df1.head()
df1.to_csv('last_item_viewed.csv', index=False)

(326663, 9)

,impression_id,last_server_time,last_device_type,last_item_id,last_item_price,last_category_1,last_category_2,last_category_3,last_product_type
5914074,00003e3b9e5336685200ae85d21b4f5e,2018-11-14 21:34:00,android,2121,249.0,0.0,45.0,117.0,6379.0
10284641,000053b1e684c9e7ea73727b2238ce18,2018-11-14 22:48:00,android,93947,4416.0,13.0,18.0,5.0,2088.0
804681,00005d011db80a956aab176cc94d1d37,2018-12-10 06:15:00,android,425,1025.0,4.0,74.0,292.0,4524.0
2679242,0000b2815cc3c2b56867cbbf4d36efa5,2018-12-05 10:22:00,android,30469,1843.0,13.0,58.0,273.0,420.0
7843712,000109e6bf54b3b20e8547b6026bc355,2018-11-20 19:38:00,android,8860,844.0,11.0,70.0,322.0,4129.0


In [25]:
# df1 = df_[df_.time_delta_in_days >= 7].drop_duplicates(subset=['impression_id'], keep='last')
# df1 = df1[['impression_id', 'server_time', 'device_type', 'item_id', 'item_price',
#      'category_1', 'category_2', 'category_3', 'product_type']]
# df1.columns = list(map(lambda x: 'last_7d_' + x if x != 'impression_id' else x, df1.columns ))
# df1.shape
# df1.head()
# df1.to_csv('last_item_viewed_7d.csv', index=False)

#### Historical Sessions

In [26]:
# df_y = df_.groupby(['impression_id', 'session_id']).agg({
#     'start_time': [('', 'last')],
#     'end_time': [('', 'last')],
#     'time_since_last_session': [('', 'last')],
#     'time_for_next_session': [('', 'last')],
#     'no_of_logs': [('', 'last')],
#     'duration': [('', 'last')],  
    
#     'item_id': [('count', 'nunique')],
#     'item_price': ['mean'],
    
# #     'category_1':[("mode", get_mode)],
# #     'category_2':[("mode", get_mode)],
# #     'category_3':[("mode", get_mode)],
# #     'product_type':[("mode", get_mode)],
    
# })
# df_y.columns = ['_'.join(col).strip('_') for col in df_y.columns.values]
# df_y = df_y.reset_index()
# df_y.shape
# df_y.head()


In [27]:
# df_x = df_y.groupby('impression_id').agg({
#     'session_id':['count'],
#     'no_of_logs': ['sum', 'min', 'max', 'mean', 'std'],
#     'duration': ['sum', 'min', 'max', 'mean', 'std'],
#     'time_since_last_session': ['min', 'max', 'mean', 'std'],
#     'time_for_next_session': ['min', 'max', 'mean', 'std'],
#     'item_id_count': ['min', 'max', 'mean', 'std'],
#     'item_price_mean': ['min', 'max', 'mean', 'std'],
# })
# df_x.columns = ['_'.join(col).strip('_') for col in df_x.columns.values]
# df_x = df_x.reset_index()
# df_x.shape
# df_x.columns = list(map(lambda x: 'hist_sessions_' + x if x != 'impression_id' else x, df_x.columns))
# df_x.head()

# df_x.to_csv('hist_sessions_vars.csv', index=False)

In [28]:
df_y = df_[df_.time_delta_in_days >= 7].groupby(['impression_id', 'session_id']).agg({
    'start_time': [('', 'last')],
    'end_time': [('', 'last')],
    'time_since_last_session': [('', 'last')],
    'time_for_next_session': [('', 'last')],
    'no_of_logs': [('', 'last')],
    'duration': [('', 'last')],  
    
    'item_id': [('count', 'nunique')],
    'item_price': ['mean'],
    
#     'category_1':[("mode", get_mode)],
#     'category_2':[("mode", get_mode)],
#     'category_3':[("mode", get_mode)],
#     'product_type':[("mode", get_mode)],
    
})
df_y.columns = ['_'.join(col).strip('_') for col in df_y.columns.values]
df_y = df_y.reset_index()
df_y.shape
df_y.head()


(3486993, 10)

,impression_id,session_id,start_time,end_time,time_since_last_session,time_for_next_session,no_of_logs,duration,item_id_count,item_price_mean
0,00003e3b9e5336685200ae85d21b4f5e,33051,2018-11-05 01:51:00,2018-11-05 01:51:00,17940.0,77460.0,1,0,1,1670.000000
1,00003e3b9e5336685200ae85d21b4f5e,322712,2018-11-02 20:52:00,2018-11-02 20:52:00,0.0,17940.0,1,0,1,544.000000
2,000053b1e684c9e7ea73727b2238ce18,421868,2018-11-14 22:48:00,2018-11-14 22:48:00,80100.0,80520.0,1,0,1,4416.000000
3,000053b1e684c9e7ea73727b2238ce18,560262,2018-11-12 00:32:00,2018-11-12 00:33:00,0.0,80100.0,2,60,2,12227.000000
4,00005d011db80a956aab176cc94d1d37,86911,2018-11-16 12:44:00,2018-11-16 13:45:00,57000.0,2700.0,22,3660,21,8118.954545


In [29]:
df_x = df_y.groupby('impression_id').agg({
    'session_id':['count'],
    'no_of_logs': ['sum', 'min', 'max', 'mean', 'std'],
    'duration': ['sum', 'min', 'max', 'mean', 'std'],
    'time_since_last_session': ['min', 'max', 'mean', 'std'],
    'time_for_next_session': ['min', 'max', 'mean', 'std'],
    'item_id_count': ['min', 'max', 'mean', 'std'],
    'item_price_mean': ['min', 'max', 'mean', 'std'],
})
df_x.columns = ['_'.join(col).strip('_') for col in df_x.columns.values]
df_x = df_x.reset_index()
df_x.shape
df_x.columns = list(map(lambda x: 'hist_sessions_7d_' + x if x != 'impression_id' else x, df_x.columns))
df_x.head()

df_x.to_csv('hist_sessions_vars_7d.csv', index=False)

(299375, 28)

,impression_id,hist_sessions_7d_session_id_count,hist_sessions_7d_no_of_logs_sum,hist_sessions_7d_no_of_logs_min,hist_sessions_7d_no_of_logs_max,hist_sessions_7d_no_of_logs_mean,hist_sessions_7d_no_of_logs_std,hist_sessions_7d_duration_sum,hist_sessions_7d_duration_min,hist_sessions_7d_duration_max,hist_sessions_7d_duration_mean,hist_sessions_7d_duration_std,hist_sessions_7d_time_since_last_session_min,hist_sessions_7d_time_since_last_session_max,hist_sessions_7d_time_since_last_session_mean,hist_sessions_7d_time_since_last_session_std,hist_sessions_7d_time_for_next_session_min,hist_sessions_7d_time_for_next_session_max,hist_sessions_7d_time_for_next_session_mean,hist_sessions_7d_time_for_next_session_std,hist_sessions_7d_item_id_count_min,hist_sessions_7d_item_id_count_max,hist_sessions_7d_item_id_count_mean,hist_sessions_7d_item_id_count_std,hist_sessions_7d_item_price_mean_min,hist_sessions_7d_item_price_mean_max,hist_sessions_7d_item_price_mean_mean,hist_sessions_7d_item_price_mean_std
0,00003e3b9e5336685200ae85d21b4f5e,2,2,1,1,1.000000,0.000000,0,0,0,0.000000,0.000000,0.0,17940.0,8970.000000,12685.495654,17940.0,77460.0,47700.000000,42086.995616,1,1,1.000000,0.000000,544.0,1670.000000,1107.000000,796.202236
1,000053b1e684c9e7ea73727b2238ce18,2,3,1,2,1.500000,0.707107,60,0,60,30.000000,42.426407,0.0,80100.0,40050.000000,56639.253173,80100.0,80520.0,80310.000000,296.984848,1,2,1.500000,0.707107,4416.0,12227.000000,8321.500000,5523.211068
2,00005d011db80a956aab176cc94d1d37,7,39,1,22,5.571429,7.546680,7380,0,3660,1054.285714,1387.742257,0.0,76920.0,35974.285714,33636.780889,2700.0,76920.0,46217.142857,31718.456096,1,21,4.000000,7.505553,352.0,8118.954545,2839.585343,2824.638381
3,0000b2815cc3c2b56867cbbf4d36efa5,10,21,1,7,2.100000,1.969207,900,0,540,90.000000,174.928557,0.0,85260.0,29574.000000,31248.775336,780.0,85260.0,34476.000000,29910.241724,1,3,1.500000,0.707107,723.0,40642.000000,15720.460714,15054.792228
4,000109e6bf54b3b20e8547b6026bc355,9,20,1,9,2.222222,2.635231,2220,0,1740,246.666667,582.065288,0.0,79620.0,32626.666667,24469.258264,0.0,79620.0,32626.666667,24469.258264,1,4,1.666667,1.118034,844.0,53760.000000,17990.333333,18008.679697


#### USER Features

In [30]:
df.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
104312,5f98c2c31a8006e510448c02ec74d50f,2018-11-26 23:30:00,0,207,old,0,0.0
242899,ccf6d380a63293580f2247d840fca638,2018-12-13 19:49:00,1,386,latest,1,NaN
49515,2b12c0d47f5821a5adb3bfd973d0f708,2018-11-19 20:49:00,2,190,intermediate,0,0.0
54202,f85e4bf0e34f3ee66add56229845e4db,2018-11-20 20:29:00,2,190,intermediate,0,0.0
24562,010ed37e44e2fdc175b4c5c6c930805a,2018-11-20 20:53:00,2,190,intermediate,0,0.0


In [31]:
df_x = df.groupby('user_id').agg({
    'impression_id': [('nunique', 'nunique')],
    'app_code': [('nunique', 'nunique'), ('mode', get_mode), ('mode_count', get_mode_count)],
    'os_version': [('mode', get_mode)],
    'is_4G': ['mean', 'sum'],
    'is_click': ['mean', 'sum'],
})

df_x.columns = ['_'.join(col).strip('_') for col in df_x.columns.values]
df_x = df_x.reset_index()
df_x.shape
df_x.head()

(89157, 10)

,user_id,impression_id_nunique,app_code_nunique,app_code_mode,app_code_mode_count,os_version_mode,is_4G_mean,is_4G_sum,is_click_mean,is_click_sum
0,0,1,1,207,1,old,0.0,0,0.0,0.0
1,1,1,1,386,1,latest,1.0,1,NaN,0.0
2,2,4,1,190,4,intermediate,0.0,0,0.0,0.0
3,3,1,1,371,1,latest,1.0,1,0.0,0.0
4,4,2,1,207,2,old,0.0,0,NaN,0.0


In [32]:
view_log.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type,start_time,end_time,no_of_logs,duration,time_since_last_session,time_for_next_session
0,2018-10-18 14:27:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
1,2018-10-18 14:43:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
2,2018-10-18 14:46:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
3,2018-10-18 14:47:00,android,268465,0,44459,3776.0,14.0,29.0,199.0,7363.0,2018-10-18 14:27:00,2018-10-18 14:47:00,4,1200,NaN,68460.0
4,2018-10-22 09:48:00,android,295495,0,40228,332.0,7.0,24.0,126.0,8351.0,2018-10-22 09:48:00,2018-10-22 09:48:00,1,0,68460.0,83940.0


In [33]:
df_y = view_log.groupby('user_id').agg({
    'session_id': [ 'count', 'nunique'],
    'item_id': ['nunique', ('mode', get_mode), ('mode_count', get_mode_count)],
    'item_price':['min', 'max', 'mean','std', ('range', lambda x : x.max() - x.min())],
    
    'device_type': [('mode', get_mode)],
    'category_1': [('mode', get_mode), 'nunique'],
    'category_2': [('mode', get_mode), 'nunique'],
    'category_3': [('mode', get_mode), 'nunique'],
    'product_type': [('mode', get_mode), 'nunique'],
    
})

df_y.columns = ['_'.join(col).strip('_') for col in df_y.columns.values]
df_y = df_y.reset_index()
df_y.shape
df_y.head()

(89157, 20)

,user_id,session_id_count,session_id_nunique,item_id_nunique,item_id_mode,item_id_mode_count,item_price_min,item_price_max,item_price_mean,item_price_std,item_price_range,device_type_mode,category_1_mode,category_1_nunique,category_2_mode,category_2_nunique,category_3_mode,category_3_nunique,product_type_mode,product_type_nunique
0,0,37,11,18,60603,5,332.0,92160.0,7905.837838,17958.103557,91828.0,android,17.0,9,49.0,15,60.0,16,2823.0,18
1,1,8,3,8,126431,1,383.0,12595.0,3946.750000,4412.414232,12212.0,android,11.0,7,76.0,7,147.0,7,10112.0,8
2,2,163,37,130,127236,6,128.0,281536.0,14289.184049,31436.352164,281408.0,android,1.0,15,42.0,47,279.0,82,8127.0,126
3,3,8,1,3,27934,3,537.0,16640.0,7257.375000,7805.928863,16103.0,android,14.0,3,62.0,3,112.0,3,491.0,3
4,4,2,1,2,38231,1,1977.0,58252.0,30114.500000,39792.434111,56275.0,android,14.0,2,61.0,2,159.0,2,4426.0,2


In [34]:
df_ = df_x.merge(df_y)
df_.shape
df_.columns = list(map(lambda x: 'user_feats_' + x if x != 'user_id' else x, df_.columns))
df_.head()

(89157, 29)

,user_id,user_feats_impression_id_nunique,user_feats_app_code_nunique,user_feats_app_code_mode,user_feats_app_code_mode_count,user_feats_os_version_mode,user_feats_is_4G_mean,user_feats_is_4G_sum,user_feats_is_click_mean,user_feats_is_click_sum,user_feats_session_id_count,user_feats_session_id_nunique,user_feats_item_id_nunique,user_feats_item_id_mode,user_feats_item_id_mode_count,user_feats_item_price_min,user_feats_item_price_max,user_feats_item_price_mean,user_feats_item_price_std,user_feats_item_price_range,user_feats_device_type_mode,user_feats_category_1_mode,user_feats_category_1_nunique,user_feats_category_2_mode,user_feats_category_2_nunique,user_feats_category_3_mode,user_feats_category_3_nunique,user_feats_product_type_mode,user_feats_product_type_nunique
0,0,1,1,207,1,old,0.0,0,0.0,0.0,37,11,18,60603,5,332.0,92160.0,7905.837838,17958.103557,91828.0,android,17.0,9,49.0,15,60.0,16,2823.0,18
1,1,1,1,386,1,latest,1.0,1,NaN,0.0,8,3,8,126431,1,383.0,12595.0,3946.750000,4412.414232,12212.0,android,11.0,7,76.0,7,147.0,7,10112.0,8
2,2,4,1,190,4,intermediate,0.0,0,0.0,0.0,163,37,130,127236,6,128.0,281536.0,14289.184049,31436.352164,281408.0,android,1.0,15,42.0,47,279.0,82,8127.0,126
3,3,1,1,371,1,latest,1.0,1,0.0,0.0,8,1,3,27934,3,537.0,16640.0,7257.375000,7805.928863,16103.0,android,14.0,3,62.0,3,112.0,3,491.0,3
4,4,2,1,207,2,old,0.0,0,NaN,0.0,2,1,2,38231,1,1977.0,58252.0,30114.500000,39792.434111,56275.0,android,14.0,2,61.0,2,159.0,2,4426.0,2


In [35]:
df_.to_csv('user_features.csv', index=False)

#### Historical Impression Logs

In [44]:
# time since last impression by the user
df['time_since_last_impression'] = ((df.impression_time.shift(-1) - df.impression_time) * (df.user_id == df.user_id.shift(-1)).astype(int)).dt.seconds
df['time_for_next_impression'] = ((df.impression_time.shift(-1) - df.impression_time) * (df.user_id == df.user_id.shift(-1)).astype(int)).dt.seconds

In [45]:
def worker(row):
    _dict ={}
    _dict['impression_id'] = row['impression_id']
    _dict['time_since_last_impression'] = row['time_since_last_impression']
    _dict['time_for_next_impression'] = row['time_for_next_impression']
    
    # previous impressions
    imp_logs = df[(df.user_id == row['user_id'] )& (df.impression_time < row['impression_time'])]   
    _dict['hist_imp_counts'] = imp_logs.shape[0]
    _dict['hist_imp_for_same_app_code_sum'] = (imp_logs.app_code == row['app_code']).sum()
    _dict['hist_imp_for_same_app_code_mean'] = (imp_logs.app_code == row['app_code']).mean()
    _dict['hist_imp_for_same_os_version_sum'] = (imp_logs.os_version == row['os_version']).sum()
    _dict['hist_imp_for_same_os_version_mean'] = (imp_logs.os_version == row['os_version']).mean()
    _dict['hist_imp_is_4G_sum'] = imp_logs['is_4G'].sum()
    _dict['hist_imp_is_4G_mean'] = imp_logs['is_4G'].mean()
    
    _dict.update(parse_dict_of_dict({
       'hist_imp': imp_logs[['time_since_last_impression', 'time_for_next_impression']].agg(['mean', 'min', 'max', 'std']).to_dict()
    }))
    
    # previous impressions last 7 days
    time_filter = pd.to_datetime((row['impression_time'] - pd.Timedelta(days=7)).date())
    imp_logs = df[(df.user_id == row['user_id'] )& (df.impression_time < row['impression_time'])
                  & (df.impression_time >= time_filter)]
    
    _dict['hist_7_days_imp_counts'] = imp_logs.shape[0]
    _dict['hist_7_days_imp_for_same_app_code_sum'] = (imp_logs.app_code == row['app_code']).sum()
    _dict['hist_7_days_imp_for_same_app_code_mean'] = (imp_logs.app_code == row['app_code']).mean()
    _dict['hist_7_days_imp_for_same_os_version_sum'] = (imp_logs.os_version == row['os_version']).sum()
    _dict['hist_7_days_imp_for_same_os_version_mean'] = (imp_logs.os_version == row['os_version']).mean()
    _dict['hist_7_days_imp_is_4G_sum'] = imp_logs['is_4G'].sum()
    _dict['hist_7_days_imp_is_4G_mean'] = imp_logs['is_4G'].mean()
    
    _dict.update(parse_dict_of_dict({
       'hist_7_days_imp': imp_logs[['time_since_last_impression', 'time_for_next_impression']].agg(['mean', 'min', 'max', 'std']).to_dict()
    }))
    
    # previous impressions last 30 days
    time_filter = pd.to_datetime((row['impression_time'] - pd.Timedelta(days=30)).date())
    imp_logs = df[(df.user_id == row['user_id'] )& (df.impression_time < row['impression_time'])
                  & (df.impression_time >= time_filter)]
    
    _dict['hist_30_days_imp_counts'] = imp_logs.shape[0]
    _dict['hist_30_days_imp_for_same_app_code_sum'] = (imp_logs.app_code == row['app_code']).sum()
    _dict['hist_30_days_imp_for_same_app_code_mean'] = (imp_logs.app_code == row['app_code']).mean()
    _dict['hist_30_days_imp_for_same_os_version_sum'] = (imp_logs.os_version == row['os_version']).sum()
    _dict['hist_30_days_imp_for_same_os_version_mean'] = (imp_logs.os_version == row['os_version']).mean()
    _dict['hist_30_days_imp_is_4G_sum'] = imp_logs['is_4G'].sum()
    _dict['hist_30_days_imp_is_4G_mean'] = imp_logs['is_4G'].mean()
    
    _dict.update(parse_dict_of_dict({
       'hist_30_days_imp': imp_logs[['time_since_last_impression', 'time_for_next_impression']].agg(['mean', 'min', 'max', 'std']).to_dict()
    }))
    

    return _dict
    

In [47]:
start_time = time.time()
argss = [(row) for i, row in df.iterrows()]
pool = mp.Pool(mp.cpu_count())
res = pool.map(worker, argss)
end_time = time.time() - start_time
print("--- %s seconds ---" % (end_time))

--- 2943.3071818351746 seconds ---


In [48]:
df_ = pd.DataFrame.from_records(res)

In [49]:
df_.to_csv('hist_imp_variables.csv', index=False)

In [50]:
df_.head()

,impression_id,time_since_last_impression,time_for_next_impression,hist_imp_counts,hist_imp_for_same_app_code_sum,hist_imp_for_same_app_code_mean,hist_imp_for_same_os_version_sum,hist_imp_for_same_os_version_mean,hist_imp_is_4G_sum,hist_imp_is_4G_mean,hist_imp_time_since_last_impression_mean,hist_imp_time_since_last_impression_min,hist_imp_time_since_last_impression_max,hist_imp_time_since_last_impression_std,hist_imp_time_for_next_impression_mean,hist_imp_time_for_next_impression_min,hist_imp_time_for_next_impression_max,hist_imp_time_for_next_impression_std,hist_7_days_imp_counts,hist_7_days_imp_for_same_app_code_sum,hist_7_days_imp_for_same_app_code_mean,hist_7_days_imp_for_same_os_version_sum,hist_7_days_imp_for_same_os_version_mean,hist_7_days_imp_is_4G_sum,hist_7_days_imp_is_4G_mean,hist_7_days_imp_time_since_last_impression_mean,hist_7_days_imp_time_since_last_impression_min,hist_7_days_imp_time_since_last_impression_max,hist_7_days_imp_time_since_last_impression_std,hist_7_days_imp_time_for_next_impression_mean,hist_7_days_imp_time_for_next_impression_min,hist_7_days_imp_time_for_next_impression_max,hist_7_days_imp_time_for_next_impression_std,hist_30_days_imp_counts,hist_30_days_imp_for_same_app_code_sum,hist_30_days_imp_for_same_app_code_mean,hist_30_days_imp_for_same_os_version_sum,hist_30_days_imp_for_same_os_version_mean,hist_30_days_imp_is_4G_sum,hist_30_days_imp_is_4G_mean,hist_30_days_imp_time_since_last_impression_mean,hist_30_days_imp_time_since_last_impression_min,hist_30_days_imp_time_since_last_impression_max,hist_30_days_imp_time_since_last_impression_std,hist_30_days_imp_time_for_next_impression_mean,hist_30_days_imp_time_for_next_impression_min,hist_30_days_imp_time_for_next_impression_max,hist_30_days_imp_time_for_next_impression_std
0,5f98c2c31a8006e510448c02ec74d50f,0.0,0.0,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan
1,ccf6d380a63293580f2247d840fca638,0.0,0.0,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan
2,2b12c0d47f5821a5adb3bfd973d0f708,85200.0,85200.0,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan,0,0,NaN,0,NaN,0,NaN,nan,nan,nan,nan,nan,nan,nan,nan
3,f85e4bf0e34f3ee66add56229845e4db,1440.0,1440.0,1,1,1.0,1,1.0,0,0.0,85200.0,85200.0,85200.0,nan,85200.0,85200.0,85200.0,nan,1,1,1.0,1,1.0,0,0.0,85200.0,85200.0,85200.0,nan,85200.0,85200.0,85200.0,nan,1,1,1.0,1,1.0,0,0.0,85200.0,85200.0,85200.0,nan,85200.0,85200.0,85200.0,nan
4,010ed37e44e2fdc175b4c5c6c930805a,3240.0,3240.0,2,2,1.0,2,1.0,0,0.0,43320.0,1440.0,85200.0,59227.263992185224,43320.0,1440.0,85200.0,59227.263992185224,2,2,1.0,2,1.0,0,0.0,43320.0,1440.0,85200.0,59227.263992185224,43320.0,1440.0,85200.0,59227.263992185224,2,2,1.0,2,1.0,0,0.0,43320.0,1440.0,85200.0,59227.263992185224,43320.0,1440.0,85200.0,59227.263992185224


In [51]:
# df_.nunique()

In [52]:
# df.head()

#### Click Variables

In [53]:
# is_click variables
def worker(row):
    
    _dict = {}
    _dict['impression_id'] = row['impression_id']
    
    time_filter = pd.to_datetime((row['impression_time']).date())
    
    _dict["hist_all_is_click"] = df[(df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_all_user_is_click"] = df[(df.user_id == row['user_id']) 
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_all_app_is_click"] = df[(df.app_code == row['app_code'] ) 
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_all_user_app_is_click"] = df[(df.app_code == row['app_code']) & (df.user_id == row['user_id'])  
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    
    
    time_filter = pd.to_datetime((row['impression_time'] - pd.Timedelta(days=7)).date())
    
    _dict["hist_is_click"] = df[(df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_user_is_click"] = df[(df.user_id == row['user_id']) 
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_app_is_click"] = df[(df.app_code == row['app_code'] ) 
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_user_app_is_click"] = df[(df.app_code == row['app_code']) & (df.user_id == row['user_id'])  
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    
    time_filter = pd.to_datetime((row['impression_time'] - pd.Timedelta(days=7)).date())
    time_filter_1 = pd.to_datetime((row['impression_time'] - pd.Timedelta(days=14)).date())
    
    _dict["hist_last_7d_is_click"] = df[(df.impression_time < time_filter)
            & (df.impression_time >= time_filter_1)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_last_7d_user_is_click"] = df[(df.user_id == row['user_id']) 
            & (df.impression_time < time_filter) & (df.impression_time >= time_filter_1)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_last_7d_app_is_click"] = df[(df.app_code == row['app_code'] ) 
            & (df.impression_time < time_filter) & (df.impression_time >= time_filter_1)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_last_7d_user_app_is_click"] = df[(df.app_code == row['app_code']) & (df.user_id == row['user_id'])  
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    
    
    time_filter = pd.to_datetime((row['impression_time'] - pd.Timedelta(days=7)).date())
    time_filter_1 = pd.to_datetime((row['impression_time'] - pd.Timedelta(days=21)).date())
    
    _dict["hist_last_14d_is_click"] = df[(df.impression_time < time_filter)
            & (df.impression_time >= time_filter_1)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_last_14d_user_is_click"] = df[(df.user_id == row['user_id']) 
            & (df.impression_time < time_filter) & (df.impression_time >= time_filter_1)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_last_14d_app_is_click"] = df[(df.app_code == row['app_code'] ) 
            & (df.impression_time < time_filter) & (df.impression_time >= time_filter_1)].is_click.agg(['mean','sum']).to_dict()
    _dict["hist_last_14d_user_app_is_click"] = df[(df.app_code == row['app_code']) & (df.user_id == row['user_id'])  
            & (df.impression_time < time_filter)].is_click.agg(['mean','sum']).to_dict()
    
    return parse_dict_of_dict(_dict)


In [ ]:
start_time = time.time()
argss = [(row) for i, row in df.iterrows()]
pool = mp.Pool(mp.cpu_count())
res = pool.map(worker, argss)
end_time = time.time() - start_time
print("--- %s seconds ---" % (end_time))

In [ ]:
df_ = pd.DataFrame.from_records(res)

In [ ]:
df_.to_csv('more_click_vars.csv', index=False)